In [105]:
from configparser import ConfigParser
config = ConfigParser()

config.read('config.ini')
username = config.get('main1', 'username')
password = config.get('main1', 'password')
account = config.get('main1', 'account')
database = config.get('main1', 'database')
schema = config.get('main1', 'schema')
warehouse = config.get('main1', 'warehouse')
role = config.get('main1', 'role')


In [110]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import sqlalchemy as db
import pandas as pd
from sqlalchemy import text

engine=create_engine('snowflake://{username}:{password}@{account}/{database}/{schema}?warehouse={warehouse}&role={role}'.format(username = username,password = password,account = account,database = database,schema = schema,warehouse = warehouse,role = role))

connection = engine.connect()
metadata = db.MetaData()
catalog = db.Table('catalog',metadata, autoload=True, autoload_with=engine)

#Equivalent to 'SELECT * FROM catalog'
query = db.select([catalog])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet[:3]
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
print(df.head())



                id                                          file_name  \
0  R19091407247863  vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5   
1  R19091407247492  vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5   
2  R19091407248679  vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5   
3  R19091407247587  vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5   
4  R19091407248492  vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5   

   file_index img_type            time_utc  \
0          72      vil 2019-09-14 07:24:35   
1          73      vil 2019-09-14 07:24:35   
2          74      vil 2019-09-14 07:24:35   
3          75      vil 2019-09-14 07:24:35   
4          76      vil 2019-09-14 07:24:35   

                                      minute_offsets episode_id  event_id  \
0  -120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...       None       NaN   
1  -120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...       None       NaN   
2  -120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-... 

In [112]:
#select img_type,event_id,file_name,EVENT_TYPE from catalog where event_type in ['Flash Flood','Flood'] order by time_utc desc
query = db.select([catalog.columns.img_type,catalog.columns.event_id,catalog.columns.file_name,catalog.columns.event_type]).where(catalog.columns.event_type.in_(['Flash Flood','Flood'])).order_by(catalog.columns.event_type,db.desc(catalog.columns.time_utc))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df2=pd.DataFrame(ResultSet)
df2.columns = ResultSet[0].keys()
print(df2.head(20))


   img_type  event_id                                          file_name  \
0      lght    858117   lght/2019/SEVIR_LGHT_ALLEVENTS_2019_0901_1001.h5   
1       vil    858117   vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5   
2     ir069    858117  ir069/2019/SEVIR_IR069_STORMEVENTS_2019_0701_1...   
3       vis    858117   vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5   
4     ir107    858117  ir107/2019/SEVIR_IR107_STORMEVENTS_2019_0701_1...   
5      lght    855374   lght/2019/SEVIR_LGHT_ALLEVENTS_2019_0901_1001.h5   
6       vil    855374   vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5   
7       vis    855374   vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5   
8     ir069    855374  ir069/2019/SEVIR_IR069_STORMEVENTS_2019_0701_1...   
9     ir107    855374  ir107/2019/SEVIR_IR107_STORMEVENTS_2019_0701_1...   
10    ir069    857832  ir069/2019/SEVIR_IR069_STORMEVENTS_2019_0701_1...   
11     lght    857832   lght/2019/SEVIR_LGHT_ALLEVENTS_2019_0901_1001.h5   
12      vil 

In [113]:
#select c.event_id, FLOOD_CAUSE,BEGIN_LOCATION,END_LOCATION,DAMAGE_PROPERTY,DAMAGE_CROPS,c.FILE_NAME,c.FILE_INDEX,c.IMG_TYPE,sd.EVENT_TYPE
#from "WEATHER"."STORM"."STORM_DETAILS" sd 
#inner join "WEATHER"."STORM"."CATALOG"  c on sd.event_id=c.event_id where flood_cause is not null
storm_details = db.Table('storm_details', metadata, autoload=True, autoload_with=engine)
catalog = db.Table('catalog', metadata, autoload=True, autoload_with=engine)
query = db.select([storm_details.columns.event_id,storm_details.columns.flood_cause,storm_details.columns.begin_location,
                   storm_details.columns.end_location,storm_details.columns.damage_property,storm_details.columns.damage_crops,
                   catalog.columns.file_name,catalog.columns.file_index,catalog.columns.img_type,storm_details.columns.event_type]).where(storm_details.columns.flood_cause !=None)
query = query.select_from(catalog.join(storm_details, catalog.columns.event_id == storm_details.columns.event_id))
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)

,event_id,flood_cause,begin_location,end_location,damage_property,damage_crops,file_name,file_index,img_type,event_type
0,857024,Heavy Rain,CHUI CHUISCHU,VAIVO VO,0.0000,0.0000,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5,6,vis,Flash Flood
1,857803,Heavy Rain / Tropical System,GOLDEN ACRES,GOLDEN ACRES,0.0000,0.0000,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5,7,vis,Flash Flood
2,848509,Heavy Rain,ARCATA,ARCATA,0.0000,0.0000,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5,16,vis,Flash Flood
3,856108,Heavy Rain,REDWOOD FALLS,REDWOOD FALLS,0.0000,0.0000,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5,53,vis,Flood
4,846040,Heavy Rain,ALAMO,ALAMO,50000.0000,0.0000,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0901_0930.h5,58,vis,Flash Flood


In [114]:
#select count(*),FATALITY_SEX from "WEATHER"."STORM"."STORM_FATALITIES_DETAILS" group by FATALITY_SEX having FATALITY_SEX!= '' 
storm_fatalities= db.Table('storm_fatalities',metadata, autoload=True, autoload_with=engine)
query=db.select([storm_fatalities.columns.fatality_sex,db.func.count(storm_fatalities.columns.fatality_sex).label('count')]).group_by(storm_fatalities.columns.fatality_sex).where(storm_fatalities.columns.fatality_sex !=None)
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)


,fatality_sex,count
0,M,16
1,F,1
